<a href="https://colab.research.google.com/github/hikmat690/AI-programming/blob/main/bbcnewspaper555.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis (Text Classification)**
*   **Downloading Datset from Kaggle to Google Colab**
*   **Text Cleaning**
*   **BERT Model (Feature Engineering)**
*   **DL Model**

# **Importing Preprocessing Libraries**

In [ ]:
!pip install -U "tensorflow-text==2.13.*"

In [ ]:
#!pip install --quiet tensorflow_text

import re
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,precision_score,accuracy_score,confusion_matrix

import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')


stopwords.words('english')
exclude = string.punctuation

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# **Reading Data**

In [ ]:
temp_df = pd.read_csv('/content/bbc-text.csv')
df = temp_df.iloc[:1000]

In [ ]:
df.shape

(2000, 2)

# **Text Cleaning & Preprocessing**

In [ ]:

def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

#exclude = "!.,?"
def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))

stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stopwords])

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [ ]:
df['text'] = df['text'].str.lower()

df['text'] = df['text'].apply(remove_html_tags)

df['text'] = df['text'].apply(remove_url)

df['text'] = df['text'].apply(remove_punc)
df['text'] = df['text'].apply(remove_stopwords)
df['text'] = df['text'].apply(stem_words)

<ipython-input-6-ae2a43c4bac6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.lower()
<ipython-input-6-ae2a43c4bac6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(remove_html_tags)
<ipython-input-6-ae2a43c4bac6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [ ]:
df.head()

,category,text
0,tech,tv futur hand viewer home theatr system plasma...
1,business,worldcom boss left book alon former worldcom b...
2,sport,tiger wari farrel gambl leicest say rush make ...
3,sport,yead face newcastl fa cup premiership side new...
4,entertainment,ocean twelv raid box offic ocean twelv crime c...


In [ ]:
df.isnull().sum()

,0
category,0
text,0


# **Feature Engineering**

**Target Column Encoding**

In [ ]:

from sklearn.preprocessing import LabelEncoder

X = df['text']
Y = df['category']

encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

print(Y)

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
print(X_train)

[4 0 3 3 1 2 2 3 3 1 1 0 0 2 3 0 2 3 0 4 4 4 3 3 4 3 1 4 2 1 2 4 1 1 0 2 4
 1 2 0 2 3 0 3 4 1 2 2 2 0 3 2 0 0 3 2 0 3 3 0 0 3 0 3 0 4 0 1 4 0 2 0 2 3
 0 4 0 3 3 0 0 3 2 0 1 2 2 0 1 0 3 3 2 3 2 3 0 3 0 1 1 3 0 2 2 4 3 1 3 4 0
 1 3 1 3 1 1 2 3 3 1 2 0 4 0 0 4 3 2 3 4 1 3 0 0 4 1 4 3 0 0 0 4 3 4 0 3 1
 3 4 3 4 2 0 4 1 3 0 2 0 1 2 2 1 2 0 1 2 2 1 2 3 0 2 2 0 1 3 0 0 3 4 3 4 4
 3 0 3 2 1 2 0 2 0 4 2 1 2 4 0 3 0 3 4 1 0 2 2 1 3 1 2 3 1 3 3 1 3 2 2 1 4
 3 2 0 1 4 3 4 3 1 2 3 1 4 0 4 0 0 4 0 3 0 3 0 3 1 0 0 1 3 3 2 4 2 2 2 4 3
 4 2 0 4 1 4 3 3 0 0 2 0 4 3 1 3 2 4 1 0 3 1 3 4 2 4 0 0 2 2 3 0 4 1 2 4 0
 1 3 1 3 1 2 0 3 0 3 3 1 3 2 2 0 1 4 0 3 0 1 0 0 0 2 0 2 1 1 0 3 0 2 0 4 1
 0 3 3 1 0 0 2 4 1 4 2 0 3 2 0 1 4 1 4 2 3 1 0 2 0 0 2 3 4 0 0 1 1 4 3 0 1
 4 4 1 1 1 2 3 2 4 0 1 2 0 1 2 2 3 3 0 2 1 0 3 0 0 3 0 2 2 2 3 4 4 0 4 3 1
 0 2 0 1 0 2 1 2 0 4 0 3 3 0 3 3 1 3 3 3 4 0 0 0 2 3 3 0 4 0 2 3 2 4 0 3 1
 0 2 4 0 3 0 2 0 0 3 1 4 2 4 0 3 0 1 4 4 0 3 3 1 2 2 2 3 2 4 4 1 2 4 1 0 3
 3 4 0 3 3 2 3 3 0 3 1 4 

**Finetuning using Deep Learning**

In [ ]:
preprocessor = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-l-12-h-768-a-12/versions/4",trainable=True)


text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
#print (text_input)
encoder_inputs = preprocessor(text_input)
#print(encoder_inputs)
outputs = encoder(encoder_inputs)
#print(outputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
#print(pooled_output)

drop_out = tf.keras.layers.Dropout(0.2,name='dropout')(pooled_output)
output = tf.keras.layers.Dense(5,activation='softmax',name='output')(drop_out)

model=tf.keras.Model(inputs=[text_input],outputs=[output])



In [ ]:
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.utils import to_categorical

# Compile the model with AdamW optimizer
model.compile(optimizer=AdamW(learning_rate=0.001, weight_decay=0.01),  # Replace Adam with AdamW
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Convert y_train to one-hot encoding using tf.keras.utils.to_categorical
y_train = to_categorical(y_train, num_classes=5)

# Train the model
history = model.fit(X_train, y_train, epochs=2, validation_split=0.1, batch_size=32)


Epoch 1/2
23/23 [==============================] - 1270s 53s/step - loss: 1.9485 - accuracy: 0.2472 - val_loss: 1.7403 - val_accuracy: 0.1750
Epoch 2/2
23/23 [==============================] - 1235s 54s/step - loss: 1.8388 - accuracy: 0.2139 - val_loss: 1.6640 - val_accuracy: 0.2000


In [ ]:
# Extract embeddings from the fine-tuned BERT model
def extract_embeddings(text_data):
    return model.predict(text_data)

# Get the embeddings for both training and testing sets
X_train_embeddings = extract_embeddings(X_train)
X_test_embeddings = extract_embeddings(X_test)


7/7 [==============================] - 98s 14s/step


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize the Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model on the BERT embeddings
rf.fit(X_train_embeddings, y_train.argmax(axis=1))  # y_train is one-hot encoded, use argmax to get class labels

# Make predictions on the test set
y_pred = rf.predict(X_test_embeddings)

# Evaluate the Random Forest model
# Remove argmax for y_test as it is already in the correct format
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest model accuracy: {accuracy:.4f}")


Random Forest model accuracy: 0.3000
